Para las películas que tengan en su descripción “in a world”(sin importar las mayúsculas) cuales son las 10 tuplas de palabras en sus descripciones más populares?

In [ ]:
#Realizamos las importanciones
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 64 kB/s 
     |████████████████████████████████| 199 kB 82.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=cae3bfdc201f589916f3533f706a2a7cc0b964459be752408e143ec7c650310a
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
#Realizamos las importaciones para usar google drive y pyspark
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#Descargamos movies.parquet
descarga = drive.CreateFile({'id':'1YKh46_KdgSC-6UZ0mttbOqVI6tXgly-F'})
descarga.GetContentFile('movies.parquet')

In [ ]:
spark = SparkSession.builder.getOrCreate() #Obtenemos una sesion de spark
sc = spark.sparkContext #Obtenemos el contexto

In [ ]:
#Creo los rdd con los que voy a trabajar
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('movies.parquet',header=True,inferSchema=True)
moviesRdd = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
#Filtro aquellas peliculas que no tengan descripccion
moviesRdd = moviesRdd.filter(lambda x: x.overview != None).filter(lambda x: 'in a world' in x.overview.lower())

In [ ]:
#Me fijo con cuantas peliculas me estoy quedando
moviesRdd.count()

111

In [ ]:
#Para obtener las tuplas lo primero que hago es mapear mis registros de forma tal que me quede con las descripciones de cada pelicula. Luego las pongo en miniscula (ya que no tomo en cuentas las mayusculas) y spliteo para tener las palabras de las descripciones separadas
#Una vez que tengo eso hago un flat map de forma tal que a partir de cada registro me cree un registro por tupla para todas las descripciones quedando de la siguiente forma (tupla,1).
#Teniendo ya cada registro mapeado con la forma (clave,valor) donde mi clave son las tuplas hago un reduceByKey contando las veces que se repitan esas tuplas. Por ultimo lo ordeno con un takeOrdered y me quedan las tuplas
moviesRdd.map(lambda x: x.overview.lower().split()).flatMap(lambda x: [((x[w]+','+x[w+1]),1) for w in range(len(x)-1)]).reduceByKey(lambda a,b:a+b).takeOrdered(10, key=lambda x: -x[1])

[('in,a', 131),
 ('a,world', 110),
 ('world,where', 37),
 ('of,the', 34),
 ('world,of', 26),
 ('in,the', 20),
 ('to,the', 18),
 ('of,a', 15),
 ('is,a', 15),
 ('the,story', 11)]

Cuando utilizo el 

```
((x[w]+','+x[w+1]),1) for w in range(len(x)-1)
```
dentro del flatmap lo que hago es lo siguiente. En x tengo una lista de todas las palabras que tiene la descripcion de una pelicula, por lo tanto si mi descripcion tiene 5 palabras mi for ira del 0 al 4. Luego para generar mis registros lo que hago es referirme a la primera palabra con x[w] y a la siguiente con x[w+1] de forma tal que queden armadas todas las tuplas
